In [134]:
# imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
pd.options.plotting.backend = 'plotly'

## result

In [135]:
df_store = pd.read_pickle("data/df_daily.pkl")

score_ARIMA = pd.read_pickle("results/f8/ARIMA_7.pkl").set_index("store")
score_PROPHET = pd.read_pickle("results/f8/PROPHET_7.pkl").set_index("store")
score_SNAIVE = pd.read_pickle("results/f8/SNAIVE_7.pkl").set_index("store")

score_ALL = score_SNAIVE.join(score_ARIMA).join(score_PROPHET)
horizon = 7


In [138]:
# mape
score_ALL.describe().filter(items=['mean','std'], axis=0).filter(like='mape', axis=1)


,mape_SNAIVE,mape_ARIMA,mape_PROPHET
mean,0.418559,0.428039,0.338375
std,0.114154,0.092490,0.098701


In [139]:
# mase
score_ALL.describe().filter(items=['mean','std'], axis=0).filter(like='mase', axis=1)


,mase_SNAIVE,mase_ARIMA,mase_PROPHET
mean,0.898513,1.298664,0.951572
std,0.437712,0.338098,0.203832


In [140]:
fig = score_ALL[['mae_SNAIVE', 'mae_ARIMA', 'mae_PROPHET']].plot()
fig.update_xaxes(title_text='')
fig.update_yaxes(title_text='MAE')


In [141]:
fig = score_ALL[['rmse_SNAIVE', 'rmse_ARIMA', 'rmse_PROPHET']].plot()
fig.update_xaxes(title_text='')
fig.update_yaxes(title_text='rmse')


In [142]:
fig = score_ALL[['mape_SNAIVE', 'mape_ARIMA', 'mape_PROPHET']].plot()
fig.update_xaxes(title_text='')
fig.update_yaxes(title_text='MAPE')


In [143]:
fig = score_ALL[['mase_SNAIVE', 'mase_ARIMA', 'mase_PROPHET']].plot()
fig.update_xaxes(title_text='')
fig.update_yaxes(title_text='MASE')


In [144]:
sample_ls = ['store_307222', 'store_350018', 'store_566790']

y_true = [df_store.loc[df_store['store_id']==307222, 'sales'][-horizon:].values/1e6,
          df_store.loc[df_store['store_id']==350018, 'sales'][-horizon:].values/1e6,
          df_store.loc[df_store['store_id']==566790, 'sales'][-horizon:].values/1e6]
y_true=pd.DataFrame({'sales':y_true}, index=sample_ls)

sample_fc = score_ALL.filter(items=sample_ls, axis=0).filter(like='fc', axis=1).join(y_true)
sample_fc


,fc_SNAIVE,fc_ARIMA,fc_PROPHET,sales
store,,,,
store_307222,2021-01-25 17.433143 2021-01-26 19.11446...,"[48.20032137436968, 42.39733446660982, 38.4800...",ds trend yhat_lower yhat_uppe...,"[24.84854, 18.834033, 31.418629, 27.555219, 35..."
store_350018,2021-01-25 16.534347 2021-01-26 15.54449...,"[75.80788006524239, 67.12306191270103, 65.1767...",ds trend yhat_lower yhat_uppe...,"[47.7719, 38.8677, 30.6969, 47.07535, 56.7408,..."
store_566790,2021-01-25 17.134802 2021-01-26 16.46452...,"[30.710727370440367, 25.258218878968346, 22.29...",ds trend yhat_lower yhat_uppe...,"[26.1868, 19.6689, 26.766327, 34.1142, 44.4857..."


In [145]:
fig = go.Figure()

store = sample_ls[0]
x = sample_fc.loc[store,'fc_SNAIVE'].index

fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'sales'], 
                         name='Actual sales',
                         line={'color': 'grey'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_SNAIVE'].values, 
                         name='SNAIVE',
                         line={'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_ARIMA'], 
                         name='SARIMAX',
                         line={'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_PROPHET']['yhat'].values, 
                         name='PROPHET',
                         line={'dash': 'dash'},
                         ))
fig.update_layout(title='Sales Forecast ' + store)
fig.update_yaxes(title_text='Forecasts (million VND)')

fig.show()


In [146]:
fig = go.Figure()

store = sample_ls[1]
x = sample_fc.loc[store,'fc_SNAIVE'].index

fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'sales'], 
                         name='Actual sales',
                         line={'color': 'grey'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_SNAIVE'].values, 
                         name='SNAIVE',
                         line={'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_ARIMA'], 
                         name='SARIMAX',
                         line={'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_PROPHET']['yhat'].values, 
                         name='PROPHET',
                         line={'dash': 'dash'},
                         ))
fig.update_layout(title='Sales Forecast ' + store)
fig.update_yaxes(title_text='Forecasts (million VND)')

fig.show()


In [147]:
fig = go.Figure()

store = sample_ls[2]
x = sample_fc.loc[store,'fc_SNAIVE'].index

fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'sales'], 
                         name='Actual sales',
                         line={'color': 'grey'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_SNAIVE'].values, 
                         name='SNAIVE',
                         line={'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_ARIMA'], 
                         name='SARIMAX',
                         line={'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_PROPHET']['yhat'].values, 
                         name='PROPHET',
                         line={'dash': 'dash'},
                         ))
fig.update_layout(title='Sales Forecast ' + store)
fig.update_yaxes(title_text='Forecasts (million VND)')

fig.show()
